# --- Day 16: Proboscidea Volcanium ---

### https://adventofcode.com/2022/day/16

In [2]:
example: str = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""

In [80]:
def parse(inp :str) -> {str: {}}:
    lined_inp: [str] = inp.split('\n') if inp.split('\n')[-1] else inp.split('\n')[:-1]
    cleaned: [str] = [line.replace(';', '').replace(',', '') for line in lined_inp]
    picked: [[str]] = [[line.split(' ')[1],int(line.split(' ')[4].split('=')[1]), line.split(' ')[9:]]  for line in cleaned]
    dicked: {} = {line[0]:{'r':line[1], 'f':line[2]} for line in picked }
    return dicked

def evaluate(path: [str], valves: {str: {}}) -> int:
    if not len(path) == 31:
        print("path too short")
        return
    
    if not path[0]=='AA':
        print("path doesn't start at AA")
        return
    
    released_steam: int = 0
        
    for index, valve in enumerate(path):
        if valve[0] == '+':
            released_steam += valves[valve[1:3]]['r'] * (30-index)
            
    return released_steam
    
    
def next_possibles( path: [str], valves: {str: {}}) -> [str]:
    possibilities: [str] = []
    
    cur = path[-1] if path[-1][0] != '+' else path[-1][1:3]
    
    possibilities += valves[cur]['f']
    
    if not '+' + cur in path:
        possibilities += [''.join(('+' + cur))]
        
    return possibilities

In [56]:
with open('input16.txt', 'r') as this:
    inp = this.read()

In [82]:
graph: {str: {}} = parse(inp)

In [89]:
graph

{'NV': {'r': 5, 'f': ['ZV', 'CG', 'YB', 'HX', 'OY']},
 'NU': {'r': 6, 'f': ['DA', 'MA', 'OA', 'DK']},
 'VU': {'r': 0, 'f': ['PS', 'FX']},
 'JW': {'r': 0, 'f': ['AA', 'MD']},
 'RI': {'r': 0, 'f': ['OY', 'DG']},
 'DG': {'r': 9, 'f': ['TG', 'RI', 'DF', 'EV', 'KW']},
 'PH': {'r': 7, 'f': ['KW', 'OW', 'LT', 'LZ']},
 'KZ': {'r': 12, 'f': ['ET', 'QV', 'CK', 'MS']},
 'IX': {'r': 0, 'f': ['TS', 'DO']},
 'MS': {'r': 0, 'f': ['LZ', 'KZ']},
 'IL': {'r': 0, 'f': ['DO', 'ET']},
 'EJ': {'r': 20, 'f': ['AV', 'JY']},
 'DK': {'r': 0, 'f': ['NU', 'CG']},
 'YB': {'r': 0, 'f': ['NV', 'PS']},
 'OA': {'r': 0, 'f': ['YA', 'NU']},
 'DA': {'r': 0, 'f': ['NU', 'RG']},
 'KO': {'r': 0, 'f': ['AA', 'TG']},
 'RG': {'r': 4, 'f': ['DF', 'DA', 'ZV', 'MD', 'LB']},
 'MA': {'r': 0, 'f': ['AA', 'NU']},
 'OW': {'r': 0, 'f': ['DO', 'PH']},
 'KW': {'r': 0, 'f': ['DG', 'PH']},
 'DO': {'r': 14, 'f': ['IX', 'IL', 'CZ', 'OW']},
 'DF': {'r': 0, 'f': ['RG', 'DG']},
 'TG': {'r': 0, 'f': ['DG', 'KO']},
 'LB': {'r': 0, 'f': ['RG', 'FX

In [121]:
def dijkstra(valve: str, graph: {str: {}}) -> {str:int}:
    shortes_paths = {v:1000090000 for v in graph.keys()}
    shortes_paths[valve] = 0
    
    unvisited_valves = list(graph.keys())
    unvisited_valves.remove(valve)
    
    
    for foll in graph[valve]['f']:
        shortes_paths[foll] = 1
        
    while(unvisited_valves):
        val = sorted(unvisited_valves, key= lambda x:shortes_paths[x])[0]
        unvisited_valves = sorted(unvisited_valves, key= lambda x:shortes_paths[x])[1:]
        
        for foll in graph[val]['f']:
            
            shortes_paths[foll] = min(shortes_paths[val]+1,shortes_paths[foll])
        
            
        
        
            
        
        
    
    return shortes_paths

In [136]:
shortes_paths = {val:dijkstra(val, graph) for val in graph.keys()}

In [137]:
shortes_paths_value = {val:{fol:shortes_paths[val][fol]+1 for fol in shortes_paths[val] if graph[fol]['r']!=0 and shortes_paths[val][fol]!=0} for val in shortes_paths.keys() if graph[val]['r']!=0 or val=='AA'}

In [151]:
graph['RG']

{'r': 4, 'f': ['DF', 'DA', 'ZV', 'MD', 'LB']}

In [152]:
graph['PS']

{'r': 18, 'f': ['EV', 'VU', 'YB']}

In [171]:
ext_path = []
def backtracking(paths,path: [str] ,remaining: int, graph) -> int:
    global ext_path
    possibilities = [nex for nex in paths[path[-1]].keys() if  nex not in path and paths[path[-1]][nex] < remaining]
    if not possibilities:
        ext_path = path
        return 0
    win = max([(remaining-paths[path[-1]][pos])*graph[pos]['r'] + backtracking(paths, path+[pos],remaining-paths[path[-1]][pos], graph)  for pos in possibilities])
    #print(possibilities)
    #print(win)
    return win
    

In [172]:
backtracking(shortes_paths_value, ['AA'], 26, graph)

1452

In [173]:
ext_path

['AA', 'UX', 'JM', 'PS', 'FX']

In [174]:
backtracking(shortes_paths_value, ['UX', 'JM', 'PS', 'FX','AA'], 26, graph)

1142

In [175]:
1452 + 1142

2594